## Adding noise to 3D MRI

This script adds noise to the MRI and saves the noise levels in an excel file and the noise images in a directory.
You can choose to save to your computer or to an external device. To do this, just run the correct chunk.

In [1]:
%reset

In [2]:
# Import the relevant libraries
import nibabel as nib
import os
import numpy as np
from glob import glob
import pandas as pd
# Import the scipy package
import scipy
from scipy import stats
import matplotlib.pyplot as plt

### Run for Mindboggle dataset

In [ ]:
# Dataset: Define the dataset to use, choosing one of the directories: 20OASIS_SynapseWeb_brain, OASIS-TRT-20_volumes, 12HLN, NKI-TRT-20_volumes, NKI-RS-22_volumes, MMRR-21_volumes
repository = '12HLN' #20OASIS_SynapseWeb_brain #OASIS-TRT-20_volumes #12HLN #NKI-TRT-20_volumes #NKI-RS-22_volumes #MMRR-21_volumes

# Directories
inputDir = ('../input')  # Directory where you want to search for MRI files
outputDir = ('../output')
outputInputDir = outputDir + '/input/'

# output path file
extension = '.pkl'
outputFile = repository + extension
dataset_path = os.path.join(outputInputDir, outputFile)

# Path with input images
dataMRI = os.path.join(inputDir, repository)
typeImage = 't1weighted.nii.gz' #'t1weighted_brain.nii.gz' #'t1weighted.nii.gz'
pathMRI = sorted(glob(dataMRI + '/**/' + typeImage, recursive=True))
SIZE = len(pathMRI)

print(f"MRI in the path: {pathMRI}")
print(f"Size is {SIZE}")

### Run for fsatMRI datset

In [ ]:
# Dataset: define it
# T1_AXIAL # Axial_T1_SE # AX_T1_3 # AX_T1_15 # AX_T1_FLASH_(POST) # AX_T1_POST_3 # AX_T1_POST_15 # T1_AXIAL # T1_AXIAL_POST_GAD
repository = 'T1_AXIAL'

# Directories
inputDir = ('../input') 
outputDir = ('../output')
outputInputDir = outputDir + '/input/'
# Directory where you want to search for MRI files
directory = '/home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI'

# output path file
extension = '.pkl'
outputFile = repository + extension
dataset_path = os.path.join(outputInputDir, outputFile)

# name of images
input_csv = ('/home/rosammq/PROJECTS/Axioms2023/input/csv/fastMRI_brain_DICOM_' + repository + '.csv')
# Read the CSV file into a pandas DataFrame
df = pd.read_csv(input_csv, index_col=None, header=None)
# Auxiliar variable
img_list = []

# Path with input images
for row_idx, row in df.iterrows():
    file_path = directory + '/' + row.iloc[0]
    print(f"MRI in the path: {file_path}")
    img_list.append(file_path)


pathMRI = img_list
SIZE = len(img_list)
print(f"Size is {SIZE}")

### Only run the following chunk is you want to save the result in the workspace

In [5]:
# Create/check the directory to output
directory1 = noiseDir
if os.path.isdir(directory1):
    print("Directory '% s' was already created" % directory1)
else:
    try:
        os.mkdir(directory1)
    except OSError:
        print("Directory '% s' failed" % directory1)
    else:
        print("Directory '% s' created" % directory1)

# Create/check a directory for the output in this file but this will be the input in another file
directory2 = result_path
if os.path.isdir(directory2):
    print("Directory '% s' was already created" % directory2)
else:
    try:
        os.mkdir(directory2)
    except OSError:
        print("Directory '% s' failed" % directory2)
    else:
        print("Directory '% s' created" % directory2)

Directory '../output/noise_dataset' was already created
Directory '../output/noise_dataset/AX_T1_POST_3_new' created


### Only run the two followings chunk is you want to save the results in an external disc

In [36]:
# To save in hard disc:
noiseDir = "/media/rosammq/My Passport/Rosammq/noiseDataset_Axioms2023"
external_dir = os.path.join(noiseDir, repository)

In [37]:
directory3 = external_dir
if os.path.isdir(directory3):
    print("Directory '% s' was already created" % directory3)
else:
    try:
        os.mkdir(directory3)
    except OSError:
        print("Directory '% s' failed" % directory3)
    else:
        print("Directory '% s' created" % directory3)

Directory '/media/rosammq/My Passport/Rosammq/noiseDataset_Axioms2023/AX_T1_POST_15' created


### Run to create the noises

In [38]:
# Prepare the experiment parameters
NumNoiseLevels=20
NumNoiselessImages=SIZE
MaxNoiseLevel=0.4

# Generate the randomly chosen noise levels
np.random.seed(seed=1)
NoiseLevels = np.random.uniform(0,MaxNoiseLevel,(NumNoiselessImages,NumNoiseLevels))

In [39]:
df = pd.DataFrame(NoiseLevels)
df.to_excel(os.path.join(noiseDir+ '/%s/NoiseLevels_%s.xlsx' % (repository, repository)), index=False, header=False)

In [40]:
# Add Rician noise to a 3D MRI
# Inputs:
#     img3D=The noiseless 3D MRI as a numpy 3D array
#     StandardDeviation=The standard deviation of the Rician noise
#     NoiseScale=Scaling parameter for the noise
#     fixedSeed=Pseudorandom seed to be employed in the noise generation
# Output:
#     module=The noisy module 3D MRI as a numpy 3D array
def ricianDistribution(img3D, StandardDeviation, NoiseScale, fixedSeed):
    if StandardDeviation==0:
        return img3D
    np.random.seed(seed=fixedSeed)
    distributionRandom1 = np.random.normal(size = img3D.shape)
    distributionRandom2 = np.random.normal(size = img3D.shape)
    # Generate the noise
    x = StandardDeviation * NoiseScale * distributionRandom1 + img3D
    y = StandardDeviation * NoiseScale * distributionRandom2
    module = np.sqrt(pow(x, 2) + pow(y, 2))
    # Return the image with noise
    return  module

### Run to generate the noise images

In [41]:
# Loop for all noiseless images
for NdxImage in range(0, NumNoiselessImages):

  # Load the noiseless 3D MRI
  NoiselessImage = nib.load(pathMRI[NdxImage]).get_fdata()

  # The interquartile range is employed as the noise scaling parameter. This way the
  # added noise level is robust against outliers in the values of the noiseless images.
  MyIQR=scipy.stats.iqr(NoiselessImage[NoiselessImage!=0])

  # Loop for all randomly chosen noise levels
  for NdxNoise in range(0, NumNoiseLevels):

    # Apply Rician noise
    NoiseLevel=NoiseLevels[NdxImage,NdxNoise]
    NoisyImage = ricianDistribution(img3D=NoiselessImage,StandardDeviation=NoiseLevel,NoiseScale=MyIQR,fixedSeed=NdxImage*NumNoiseLevels+NdxNoise)
    img = nib.Nifti1Image(NoisyImage, np.eye(4))  # Save axis for data (just identity)
    img.to_filename(os.path.join(noiseDir+ '/%s/Image%s_Noise%s.nii.gz' % (repository, NdxImage, NdxNoise)) ) # Save as NIfTI file